In [ ]:
import re

import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

import math

import nltk

from sklearn.feature_extraction.text import CountVectorizer

from collections import defaultdict

In [ ]:
df=pd.read_csv("temp_data3.csv")
df=df.iloc[:4000,:]
df

In [ ]:
list1=[]
list1=df.Sentiment.values

In [ ]:
for i in range(len(list1)):
    if(list1[i]=="mixed"):
        df.drop([i],axis=0,inplace=True)
    elif(list1[i]=="neutral"):
        df.drop([i],axis=0,inplace=True)

df["Sentiment"].value_counts()

In [ ]:
text=df["cleaned_text"]
sentiment=df["Sentiment"]
dataf=pd.concat([text,sentiment],axis=1)
dataf

In [ ]:
data=dataf["cleaned_text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
data=data.str.replace("[^\w\s]","")#harf dışı simgeler,noktalama
data= data.str.replace(" \d+", " ")#sayılar 
from nltk.corpus import stopwords
nltk.download("stopwords")
duraklama_kelimeleri = stopwords.words()
data=pd.DataFrame(data,columns=["cleaned_text"])
data=data["cleaned_text"].apply(lambda x: " ".join(x for x in x.split() if x not in duraklama_kelimeleri))
data=pd.concat([data,sentiment],axis=1)

In [ ]:
data

In [ ]:
#nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()#kelime bölümü
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['cleaned_text'] = data.cleaned_text.apply(lemmatize_text)

In [ ]:
cleaned_text = data['cleaned_text'].values
labels = data['Sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
#for i in range(len(labels)):
    #print(labels[i]," : ",encoded_labels[i]) 

In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(cleaned_text, encoded_labels, stratify = encoded_labels)

In [ ]:
X

In [ ]:
vec = CountVectorizer()
X = vec.fit_transform(train_sentences)#yorumlarda geçen kelimelerin sayısı
vocab = vec.get_feature_names()# kelimeler tutulur
X = X.toarray()
word_counts = {}
for l in range(2):# 2 etiket
    word_counts[l] = defaultdict(lambda: 0)#sözlük içinde 2 sözlük 
for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]#her kelimenin etiketler içindeki sayıları

In [ ]:
len(vocab)

In [ ]:
word_counts

In [ ]:
data1 = {}
def group_by_label(x, y, labels):
    for l in labels:
        data1[l] = x[np.where(y == l)]# eğitim yorumları etiketlere göre yerleştirilir
    return data1

In [ ]:
data1

In [ ]:
n_label_items = {}
log_label_priors = {}
def fit(x, y, labels):#egitimyorumları, eg.etikt, etiketler
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():#yorumların etiketleri
        n_label_items[l] = len(data)# her etikette kaç yorum olduğu tutulur
        log_label_priors[l] = math.log(n_label_items[l] / n)# etiketteki yorumsayısı/ eğitim yorumsayısı
    return n_label_items, log_label_priors

In [ ]:
n_label_items, log_label_priors

In [ ]:
def laplace_smoothing(n_label_items, vocab, word_counts, word,label):
    a = word_counts[label][word] + 1 #mevcut etiketteki mevcut kelimenin sayısına eklenir
    b = n_label_items[label] + len(vocab)#etiketteki toplam yorum sayısı + kelime sayısı
    return math.log(a/b)

In [ ]:
result = []
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences):
    for text in test_sentences:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab: continue# eğitim kelimelerindense oranlarda değişiklik olmayacağı için geç
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l# olasılık guncelle
        result.append(max(label_scores, key=label_scores.get))#olasılığı en büyük olan etiket
    return result#etiket

In [ ]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)

print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred))
print("\n")

for i in range(len(pred)):
    if(pred[i] == 0):
        print(test_sentences[i]," : negative\n")
    elif(pred[i] == 1):
        print(test_sentences[i]," : positive\n")